# ALGOSUP AI PROJECT 5 D

In [31]:
!pip install -q kaggle
!pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow
import pandas
import numpy
import librosa
import os

import matplotlib.pyplot

from tensorflow import keras
from sklearn.model_selection import train_test_split
from typing import Union, Tuple
from google.colab import files

In [ ]:
files.upload()

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
  
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s "CommonVoice Langage"
!kaggle datasets download -d olmatz/commonvoice-langage
!unzip commonvoice-langage.zip
!rm -rf commonvoice-langage.zip
!rm -rf "dataset/train/chinese"
!rm -rf "dataset/train/german"
!rm -rf "dataset/train/italian"
!rm -rf "dataset/train/spanish"
!rm -rf "dataset/validation/chinese"
!rm -rf "dataset/validation/german"
!rm -rf "dataset/validation/italian"
!rm -rf "dataset/validation/spanish"

In [43]:
datapath = "dataset/"

# doc: https://www.tensorflow.org/api_docs/python/tf/keras/utils/audio_dataset_from_directory
train_ds = keras.utils.audio_dataset_from_directory(
    datapath+"train",
    validation_split=0.2,
    subset="training",
    seed=123,
    #output_sequence_length=None, # 64000
    batch_size=32
)

test_ds = keras.utils.audio_dataset_from_directory(
    datapath+"validation",
    validation_split=0.2,
    subset="validation",
    seed=123,
    #output_sequence_length=None, # 64000
    batch_size=32
)

Found 28110 files belonging to 2 classes.
Using 22488 files for training.
Found 7029 files belonging to 2 classes.
Using 1405 files for validation.


In [44]:
class_names = train_ds.class_names

for images, labels in train_ds.take(1):
  for i, image in enumerate(images):
    print(image.shape, class_names[labels[i]])

(64000, 1) french
(64000, 1) french
(64000, 1) french
(64000, 1) french
(64000, 1) english
(64000, 1) english
(64000, 1) french
(64000, 1) french
(64000, 1) english
(64000, 1) french
(64000, 1) english
(64000, 1) english
(64000, 1) french
(64000, 1) french
(64000, 1) french
(64000, 1) english
(64000, 1) french
(64000, 1) french
(64000, 1) english
(64000, 1) french
(64000, 1) english
(64000, 1) english
(64000, 1) english
(64000, 1) english
(64000, 1) english
(64000, 1) french
(64000, 1) english
(64000, 1) french
(64000, 1) french
(64000, 1) french
(64000, 1) english
(64000, 1) english


In [45]:
keras.backend.clear_session()

#### IN PROGRESS..

In [ ]:
model = keras.Sequential([
            keras.layers.Conv2D(16, (3, 3), activation="relu", input_shape=(64000, 1)),
            keras.layers.MaxPooling2D(2, 2),
            keras.layers.Conv2D(32, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D(2, 2),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.Flatten(),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer = "adam",
                    loss="binary_crossentropy",
                    metrics=["accuracy"])

model.summary()

In [ ]:
model.fit(train_ds,
          validation_data=test_ds, epochs=5)


In [5]:
SAMPLE_SIZE = 20 # in seconds

In [ ]:
def load_and_trim(path: str, min_db: int = 20) -> Tuple[numpy.ndarray, int]:
  """Load a sound from a path and trim his content"""
  sound_array, samplerate = librosa.core.load(path)
  trim, _ = librosa.effects.trim(sound_array, top_db=min_db)

  return trim, samplerate

def frame_sound(sound: numpy.ndarray, sample_rate: int, sample_size: Union[float, int] = 20) -> numpy.ndarray:
  """Return few sample from the sound of size `sample_size`"""
  return librosa.util.frame(
    sound,
    frame_length=sample_rate * sample_size,
    hop_length=sample_rate * sample_size,
    axis=0
  )

In [ ]:
train_path = os.path.join(datapath, "train")
test_path = os.path.join(datapath, "test")

EN_train_path = os.path.join(train_path, "EN_train.wav")
FR_train_path = os.path.join(train_path, "FR_train.wav")

In [ ]:
EN_data = frame_sound(*load_and_trim(EN_train_path), SAMPLE_SIZE)
FR_data = frame_sound(*load_and_trim(FR_train_path), SAMPLE_SIZE)

EN_data = EN_data[:13]

for en, fr in zip(EN_data, FR_data):
  if not len(en) == len(fr):
    raise ValueError()

print(EN_data.shape, FR_data.shape)

(13, 441000) (13, 441000)


In [ ]:
X = numpy.concatenate((EN_data, FR_data))
a = numpy.array([1 for _ in range(len(FR_data))])
b = numpy.array([2 for _ in range(len(FR_data))])
Y = numpy.concatenate((a,b))

targets = keras.utils.to_categorical(Y - 1, 2)

print(X.shape, Y.shape)
print(X)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, targets, test_size=0.1, random_state=42)

print(f"XTrain:{len(x_train)} {x_train.shape}, XTest:{len(x_test)}, YTrain{len(y_train)}, YTest{len(y_test)}")

XTrain:23 (23, 441000), XTest:3, YTrain23, YTest3


In [ ]:
keras.backend.clear_session()

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout, MaxPooling1D, Conv1D, Reshape

model = keras.models.Sequential([
                  keras.layers.Dense(units = 1024),
                  keras.layers.Dense(units = 1024),
                  keras.layers.Dense(units = 2048),
                  keras.layers.Dense(units = 2048),
                  keras.layers.Dense(units = 1024, activation = 'relu'),
                  keras.layers.Dense(units = 512, activation = 'relu'),
                  keras.layers.Dropout(0.5),
                  keras.layers.Dense(units = 2, activation = 'softmax')                     
])

loss_func = keras.losses.CategoricalCrossentropy()
model.compile(optimizer="adam", loss=loss_func, metrics=["accuracy"])

model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=25)